# Lyft: Working with Map API

This is a quick tutorial to show the different underlying concepts behind HD map from Lyft dataset. After this short tutorial, you should be able to understand the underlying concepts as well as how to query for different map layers. Feel free to contribute visualization for each layer in this notebook.

In [ ]:
!pip install --upgrade pip
!pip install pymap3d==2.1.0
!pip install -U l5kit

In [ ]:
import numpy as np
import l5kit, os
import matplotlib.pyplot as plt
from l5kit.rasterization import build_rasterizer
from l5kit.configs import load_config_data
from l5kit.visualization import draw_trajectory, TARGET_POINTS_COLOR
from l5kit.geometry import transform_points
from tqdm import tqdm
from collections import Counter
from l5kit.data import PERCEPTION_LABELS
from prettytable import PrettyTable
# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = "../input/lyft-motion-prediction-autonomous-vehicles"
# get config
cfg = load_config_data("../input/lyft-config-files/visualisation_config.yaml")

## Load The Dataset

In [ ]:
from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.dataset import EgoDataset, AgentDataset
dm = LocalDataManager()
dataset_path = dm.require(cfg["val_data_loader"]["key"])
zarr_dataset = ChunkedDataset(dataset_path)
zarr_dataset.open()

## Load the MapAPI

In [ ]:
from l5kit.data.map_api import MapAPI
from l5kit.rasterization.rasterizer_builder import _load_metadata

semantic_map_filepath = dm.require(cfg["raster_params"]["semantic_map_key"])
dataset_meta = _load_metadata(cfg["raster_params"]["dataset_meta_key"], dm)
world_to_ecef = np.array(dataset_meta["world_to_ecef"], dtype=np.float64)

map_api = MapAPI(semantic_map_filepath, world_to_ecef)

In [ ]:
MAP_LAYERS = ["junction", "node", "segment", "lane"]


def element_of_type(elem, layer_name):
    return elem.element.HasField(layer_name)


def get_elements_from_layer(map_api, layer_name):
    return [elem for elem in map_api.elements if element_of_type(elem, layer_name)]


class MapRenderer:
    
    def __init__(self, map_api):
        self._color_map = dict(drivable_area='#a6cee3',
                               road_segment='#1f78b4',
                               road_block='#b2df8a',
                               lane='#474747')
        self._map_api = map_api
    
    def render_layer(self, layer_name):
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_axes([0, 0, 1, 1])
        
    def render_lanes(self):
        all_lanes = get_elements_from_layer(self._map_api, "lane")
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_axes([0, 0, 1, 1])
        for lane in all_lanes:
            self.render_lane(ax, lane)
        return fig, ax
        
    def render_lane(self, ax, lane):
        coords = self._map_api.get_lane_coords(MapAPI.id_as_str(lane.id))
        self.render_boundary(ax, coords["xyz_left"])
        self.render_boundary(ax, coords["xyz_right"])
        
    def render_boundary(self, ax, boundary):
        xs = boundary[:, 0]
        ys = boundary[:, 1] 
        ax.plot(xs, ys, color=self._color_map["lane"], label="lane")
        
        

In [ ]:
renderer = MapRenderer(map_api)
fig, ax = renderer.render_lanes()

### There are 7 types of map elements:

- Junction
- RoadNetworkSegment
- RoadNetworkNode
- Lane
- TrafficControlElement
- SegmentSequence 
- AnnotatedShape 

Let's dive into each one.

## Junction

Junction is conceptually a "big" intersection. This can consist of multiple smaller "intersections" which could be lane merges.

In [ ]:
def is_junction(elem, map_api):
    return elem.element.HasField("junction")

def get_junctions(map_api):
    return [elem for elem in map_api.elements if is_junction(elem, map_api)]

all_junctions = get_junctions(map_api)
all_junctions[0]

## Road Network Node

Road network node are map intersection connecting several road segments. Each road network node belongs to one junction. If we think about the road network as a graph, this will be a node in the graph where a road segment would be an edge connecting two nodes.

In [ ]:
def is_node(elem, map_api):
    return elem.element.HasField("node")

def get_nodes(map_api):
    return [elem for elem in map_api.elements if is_junction(elem, map_api)]

all_nodes = get_nodes(map_api)
all_nodes[0]

## Road Network Segment

This is the drivable surface connecting two intersections (nodes). If we think about the road network as a graph, and each intersection as a node, then a road segment is an edge connecting two nodes.

In [ ]:
def is_segment(elem, map_api):
    return elem.element.HasField("segment")

def get_segments(map_api):
    return [elem for elem in map_api.elements if is_segment(elem, map_api)]

all_segments = get_segments(map_api)
all_segments[0]

## Lane

Each road segment is built from multiple lanes. It is worth noting that the structure of lane allows for one to traverse the lane graph by using lane.lanes_ahead and lane.adjacent_lane_change_left  and lane.adjacent_lane_change_right

In [ ]:
def is_lane(elem, map_api):
    return elem.element.HasField("lane")

def get_lanes(map_api):
    return [elem for elem in map_api.elements if is_lane(elem, map_api)]

all_lanes = get_lanes(map_api)
all_lanes[0]

## Traffic Control Element

Traffic signals, e.g. traffic lights or individual "faces" of traffic lights, stop signs, yield signs, etc. controlling the exit from the lane onto one of the lanes ahead.

In [ ]:
def is_traffic_element(elem, map_api):
    return elem.element.HasField("traffic_control_element")

def get_traffic_elements(map_api):
    return [elem for elem in map_api.elements if is_traffic_element(elem, map_api)]

all_traffic_elements = get_traffic_elements(map_api)
all_traffic_elements[0]

## Segment Sequence

An ordered collection of connected segments, indicating that it is either illegal, or conversely, the only option, to proceed along that path when starting from the first segment. Typically used to represent turn restrictions, with 2 or more segments in the sequence. For complicated intersections, the sequence can have more segments, with the intermediate ones just joining nodes that are all in the intersection. In some instances, it is more concise to represent the only path that can be taken, as opposed to all the paths that cannot be taken. The reason for keeping orientation  is to present the directionality; one example is where a turn restriction is on the same from and to segment, and the restriction is only on one side of the segment. With only list of segments, it's not possible to represent this restriction, as we have to distinguish one end from the other end.

But the small version of the dataset doesn't seem to have any segment sequences.

In [ ]:
def is_segment_sequence(elem, map_api):
    return elem.element.HasField("segment_sequence")

def get_segment_sequences(map_api):
    return [elem for elem in map_api.elements if is_segment_sequence(elem, map_api)]

all_segment_sequences = get_segment_sequences(map_api)
print(len(all_segment_sequences))

## Annotated Shape

Represents polygonal data that are either physical (e.g. buildings) or imaginary (e.g. political boundaries) and their associated properties.

In [ ]:
def is_annotated_shape(elem, map_api):
    return elem.element.HasField("annotated_shape")

def get_annotated_shapes(map_api):
    return [elem for elem in map_api.elements if is_annotated_shape(elem, map_api)]

all_annotated_shapes = get_annotated_shapes(map_api)
all_annotated_shapes[0]